In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import sys, os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

cd = os.path.split(os.getcwd())[0]
if cd not in sys.path:
    sys.path.append(cd)

from lib import noaa, bexarcrime

In [ ]:
class City:
    def __init__(self, name, filepath, isd_code):
        self.name = name
        self.filepath = filepath
        self.isd_code = isd_code
    
    def load_crime(self, process=True):
        self.dfc = pd.read_csv(self.filepath, compression='gzip')
        self.dfv = self.dfc.where(self.dfc.Crime.isin(['Assault', 'Robbery', 'Shooting', 'Arson'])).dropna()
        self.all_crime = self.dfc
        self.violent_crime = self.dfv
        if process: 
            return self.process_crime()
        
        return self
    
    def process_crime(self):
        self.dfc.Time = pd.to_datetime(self.dfc.Time, errors='coerce')
        self.dfc = self.dfc.set_index('Time')
        self.dfv = self.dfc.where(self.dfc.Crime.isin(['Assault', 'Robbery', 'Shooting', 'Arson'])).dropna()
        self.all_crime = self.dfc
        self.violent_crime = self.dfv
        
        return self
        
    def load_weather(self, start=2016, end=2018):
        self.dfw = noaa.noaa_from_web(self.isd_code, start, end).fillna(method='backfill')
            
        # drop relative humididty 
        self.dfw = self.dfw.drop('RHPeriod', axis = 1)
        
        # replace null values
        self.dfw['Temperature'] = self.dfw['Temperature'].replace(9999,np.nan)
        self.dfw['Pressure'] = self.dfw['Pressure'].replace(99999,np.nan)
        self.dfw['Humidity'] = self.dfw['Humidity'].replace(999, np.nan)
        self.dfw['Sky'] = self.dfw['Sky'].replace([9,99], np.nan)
        
        # scale values back
        self.dfw['Temperature'] = self.dfw['Temperature'].map(lambda x : x/10)
        self.dfw['Pressure'] = self.dfw['Pressure'].map(lambda x : x/10)
        
        # map sky oktas to coverage percentages, roughly
        self.dfw['Sky'] = self.dfw['Sky'].map(lambda x : x/8)
        
        # convert C to F
        self.dfw['Temperature'] = self.dfw['Temperature'].map(lambda x : x * 9/5 + 32)
        self.weather = self.dfw
        
        return self
        
    def merge_dfs(self, start='2016-01-01', end='2017-01-01'):
        self.df = self.dfw.join(self.dfc, how='outer')
        self.df = self.df.groupby( 
                        [self.df.index.year, 
                         self.df.index.month, 
                         self.df.index.day, 
                         self.df.index.hour]
                    ).agg ({   
                         'Temperature' : 'mean', 
                         'Pressure' : 'mean',
                         'Humidity' : 'mean',
                         'Sky' : 'mean',
                         'Crime' : 'count'}
                    ).reset_index().rename(columns={
                        'level_0':'year',
                        'level_1':'month',
                        'level_2':'day',
                        'level_3':'hour',
                    })
        s = pd.to_datetime(self.df[['year', 'month', 'day', 'hour']])
        self.df = self.df.set_index(s).drop(['year', 'month', 'day', 'hour'], 
                                    axis=1)
        
        self.df = self.df.loc[self.df.index > start]
        self.df = self.df.loc[self.df.index < end]

        return self

In [ ]:
cities = {
    ##'Albany, GA': City('Albany, GA', '../data/crime_albany_ga.csv.gz', '722160-13869'),
    ## Not enough data for Albany
    'Baltimore, MD': City('Baltimore, MD', '../data/crime_baltimore_md.csv.gz', '745944-93784'),
    'Detroit, MI': City('Detroit, MI', '../data/crime_detroit_mi.csv.gz', '725375-14822'),
    'Flint, MI': City('Flint, MI', '../data/crime_flint_mi.csv.gz', '726370-14826'),
    'Memphis, TN': City('Memphis, TN', '../data/crime_memphis_tn.csv.gz', '723340-13893'),
    'Philadelphia, PA': City('Philadelphia, PA', '../data/crime_philadelphia_pa.csv.gz', '724080-13739'),
    ##'Pine Bluff AR': 
    ## no data for Pine Bluff
    'St. Louis, MO': City('St. Louis, MO', '../data/crime_st.+louis_mo.csv.gz', '725314-03960'),
    'Toledo, OH': City('Toledo, OH', '../data/crime_toledo_oh.csv.gz', '720275-04872')
}

In [ ]:
for city in cities.values():
    print(city.name)
    city.load_weather().load_crime().merge_dfs()
    df = city.df[['Temperature', 'Crime']].dropna()
    sns.jointplot(x='Temperature', y='Crime', data=df)
    plt.title(city.name)